In [3]:
import mymodel
from transformers import AutoTokenizer, GPT2Model
import torch
#checkpoint = './testmodel3weights_2024-07-07--05:59:32'
tokenizer = AutoTokenizer.from_pretrained('gpt2')



In [5]:
checkpoint = './backmodel3_weights_2024-07-24--19:55:58alaki'
model = mymodel.MyModel.from_pretrained(checkpoint)



Some weights of MyModel were not initialized from the model checkpoint at ./backmodel3_weights_2024-07-24--19:55:58alaki and are newly initialized: ['decoder.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
second_input = 'Once upon a time, there was a big car named Dependable.'
second_input_ids = tokenizer(second_input, return_tensors='pt')['input_ids']

In [20]:
#model_state_dict = model.state_dict()
#torch.save(model_state_dict, './modelalaki')

### loading model from pydict
from transformers import AutoConfig, AutoModel
import safetensors
#state_dict = torch.load('./nnmodel3_weights_2024-07-16--01:57:44')
state_dict = torch.load('./nnmodel3_weights_2024-07-17--04:24:10')
#state_dict = torch.load('./nnmodel3_weights_2024-07-24--16:22:27alaki')


#state_dict = torch.load('./nnmodel3_default_outputs_2024-07-17--04:24:10/checkpoint-30000/model.safetensors')

#mymodel = safetensors.torch.load('./nnmodel3_default_outputs_2024-07-17--04:24:10/checkpoint-30000/model.safetensors')

#mymodel3  = AutoModel.from_pretrained('./nnmodel3_default_outputs_2024-07-17--04:24:10/checkpoint-30000')

myconfig = AutoConfig.from_pretrained('gpt2')
myconfig2 = AutoConfig.from_pretrained('gpt2')
context_length=512
myconfig.n_ctx = context_length
model2 = mymodel.MyModel(myconfig)
model2.load_state_dict(state_dict)
randmodel = mymodel.MyModel(myconfig2)


In [23]:
input_str = '''Once upon a time, there was a big car named Dependable. He had a very important job. Dependable would take a family to the park every day. The family had a mom, dad, and a little girl named Lily. They all had a lot of love for each other.\n\nOne day, when they got to the park, they saw a big sign that said, "Fun Race Today!" The family was very excited. They knew that Dependable was very fast and could win the race. So, they decided to join the race.\n\nThe race started, and Dependable went very fast. The other cars tried to catch up, but Dependable was too quick. In the end, Dependable won the race! The family was so happy and proud of their car. They knew that their love for each other and their trust in Dependable made them win the race. And from that day on, they had even more fun at the park, knowing that they had the fastest and most dependable car around.'''
input_ids = tokenizer(input_str, return_tensors='pt')['input_ids']
hidden_embedding = model2.encoder(input_ids).last_hidden_state.detach()[0, -1, :].unsqueeze(0)

In [14]:

## don't run this!
hidden_embedding = model2.encoder(torch.tensor(input_ids)).last_hidden_state.detach()[0, -1, :].unsqueeze(0)

## now create the new input to the decoder, first obtain the word embeddings using a pretrained gpt2 model
second_input = 'Once upon a time, there was an elderly farmer, who had '
second_input_ids = tokenizer(second_input, return_tensors='pt')['input_ids']
decoder_hidden_inputs = model2.second_encoder(torch.tensor(second_input_ids), output_hidden_states=True).hidden_states[0][0].detach()
#print('shape is ', decoder_hidden_outputs.shape)

#second_input_hidden_embeddings = model.second_encoder(tokenizer(second_input, return_tensors='pt')['input_ids']).last_hidden_states[0][0]

updated_input = torch.cat((hidden_embedding, decoder_hidden_inputs), dim=0).unsqueeze(0)
updated_input.shape
#torch.zeros((439, 768))
#model.decoder()

/tmp/ipykernel_873995/26876073.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hidden_embedding = model2.encoder(torch.tensor(input_ids)).last_hidden_state.detach()[0, -1, :].unsqueeze(0)
/tmp/ipykernel_873995/26876073.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_hidden_inputs = model2.second_encoder(torch.tensor(second_input_ids), output_hidden_states=True).hidden_states[0][0].detach()


torch.Size([1, 15, 768])

In [2]:
#import safetensors.torch 
#myfile = safetensors.torch.load_file('./testmodel3weights_2024-07-07--05:59:32/model.safetensors')
#print('keys are ', myfile.keys())
#for item in list(myfile.keys()):
#    if 'lm_head' in item:
#        print(item)

keys are  dict_keys(['decoder.transformer.h.0.attn.c_attn.bias', 'decoder.transformer.h.0.attn.c_attn.weight', 'decoder.transformer.h.0.attn.c_proj.bias', 'decoder.transformer.h.0.attn.c_proj.weight', 'decoder.transformer.h.0.ln_1.bias', 'decoder.transformer.h.0.ln_1.weight', 'decoder.transformer.h.0.ln_2.bias', 'decoder.transformer.h.0.ln_2.weight', 'decoder.transformer.h.0.mlp.c_fc.bias', 'decoder.transformer.h.0.mlp.c_fc.weight', 'decoder.transformer.h.0.mlp.c_proj.bias', 'decoder.transformer.h.0.mlp.c_proj.weight', 'decoder.transformer.h.1.attn.c_attn.bias', 'decoder.transformer.h.1.attn.c_attn.weight', 'decoder.transformer.h.1.attn.c_proj.bias', 'decoder.transformer.h.1.attn.c_proj.weight', 'decoder.transformer.h.1.ln_1.bias', 'decoder.transformer.h.1.ln_1.weight', 'decoder.transformer.h.1.ln_2.bias', 'decoder.transformer.h.1.ln_2.weight', 'decoder.transformer.h.1.mlp.c_fc.bias', 'decoder.transformer.h.1.mlp.c_fc.weight', 'decoder.transformer.h.1.mlp.c_proj.bias', 'decoder.transfo

In [ ]:
#index = 0
#for param in model.encoder.parameters():
#    if index  > 3:
#        print('param is ', param)
#    index += 1

In [64]:
#from transformers import GPT2LMHeadModel, AutoConfig
#print(updated_input.shape)
#decoder_second = GPT2LMHeadModel(myconfig)

#model.decoder.output_hidden_states = True
#outputs = model2.decoder(input_ids=tokenizer(input_str, return_tensors='pt')['input_ids'])#,  output_hidden_states=True
#, attention_mask=tokenizer(input_str, return_tensors='pt')['attention_mask']

#print('output logits are ', outputs)
#strr = model2.decoder.generate(inputs_embeds = updated_input)
#print('strr is ', tokenizer.decode(strr[0]))

logits = model2.decoder(inputs_embeds = updated_input)['logits']
#model2(input_embeds = )
max_index = torch.argmax(logits, dim=2)
#max_index
#logits.shape
decoded_seq = tokenizer.decode(max_index[0])
decoded_seq
#print('logits are ', logits[2].sum())


' grand upon a time, there was a elderly couple, with happily been became'

In [10]:
second_input = 'Once upon a time, there was a big car named Dependable.'
second_input_ids = tokenizer(second_input, return_tensors='pt')['input_ids']
print(second_input_ids)
for i in range(300):
    #second_input = 'Once upon a time, there was an elderly farmer'
    
    decoder_hidden_inputs = model2.second_encoder(torch.tensor(second_input_ids), output_hidden_states=True).hidden_states[0][0].detach()
    #print('shape is ', decoder_hidden_outputs.shape)
    #second_input_hidden_embeddings = model.second_encoder(tokenizer(second_input, return_tensors='pt')['input_ids']).last_hidden_states[0][0]
    updated_input = torch.cat((hidden_embedding, decoder_hidden_inputs), dim=0).unsqueeze(0)
    #updated_input = decoder_hidden_inputs.unsqueeze(0)
    logits = model2.decoder(inputs_embeds = updated_input)['logits']
    #model2(input_embeds = )
    max_index = torch.argmax(logits, dim=2)
    #max_index
    #logits.shape
    
    #outword = tokenizer.decode(max_index[0][-1])
    second_input_ids = torch.cat((second_input_ids, max_index[:,-1:]),  dim=1)
    #print('second input is ', second_input)
#print(second_input_ids)
second_input = tokenizer.decode(second_input_ids[0])
second_input

tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,  1263,  1097,
          3706, 37947,   540,    13]])


/tmp/ipykernel_990609/1929015111.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  decoder_hidden_inputs = model2.second_encoder(torch.tensor(second_input_ids), output_hidden_states=True).hidden_states[0][0].detach()


'Once upon a time, there was a big car named Dependable. Dependable had a very important job. He liked to go fast. Today, he was going to race with the other cars.\n\nThe race began, and Dependable got very happy. He said to a car named Car, "Let\'s race, Car!" Car said, "OK, let\'s go!" They started to race and the dependable was very big.\n\nAfter the race, Dependable knew that racing would be fun. The race should finish, but Truck did not want to lose. They raced again, but the race starting.\n\nThe race began, and the dependable was very fast. But then, Car\'s engine broke. The car was in danger of surrendering to its friend, Truckable. Truck was brave and stopped the race. He remembered how dependable and good Car was at the end. So, he won the race with his engine in his heart. The car with its engine was happy and cheered for his friend. They knew that Dependable could win the race and support their friend, because they were there for their friend. Everyone learned from Dependab

In [17]:
updated_input

tensor([[-1.9310e+00,  1.6959e+00, -2.0372e-01,  ..., -1.0425e+00,
         -2.0197e+00, -1.9180e+00],
        [ 7.1667e-02, -1.1264e-02, -3.1603e-02,  ...,  5.5578e-02,
          0.0000e+00, -2.1566e-02],
        [-2.8862e-02,  0.0000e+00, -1.2996e-03,  ...,  4.9011e-02,
         -5.8250e-03,  1.3068e-02],
        ...,
        [ 3.3293e-02,  3.9718e-02, -1.8351e-02,  ..., -4.3628e-02,
         -3.6502e-02, -2.9447e-02],
        [-2.9202e-06, -1.7530e-02, -1.8004e-02,  ...,  7.1160e-03,
         -3.1151e-03, -3.2737e-02],
        [-2.0993e-02, -7.1361e-02,  3.9832e-02,  ...,  1.0748e-02,
         -5.2369e-02,  2.5638e-02]])

In [5]:
from transformers import AutoTokenizer, GPT2LMHeadModel
import torch
from datasets import load_dataset
tinystories =  load_dataset("roneneldan/TinyStories")
tinystories
from mymodelnew3 import MyModel
checkpoint = './posmodel3_weights_2024-08-01--05:36:40alaki'
#checkpoint = './posmodel3_weights_2024-07-31--00:22:21alaki'
#checkpoint = './testmodel3weights_2024-07-07--05:59:32'
#checkpoint = './encoder_is_pretrained_average.py_gpt2tokenizer_2024-07-14--21:42:14/model3weights'
model2 = MyModel.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained('gpt2')

Repo card metadata block was not found. Setting CardData to empty.


In [16]:
from datasets import load_from_disk
import numpy as np
tokenized_datasets = load_from_disk('./model3-outputs-gpt2tokenizer-varlength')
## slicing the dataset to a smaller size

tokenized_datasets = {
    'train': tokenized_datasets['train'].select(np.arange(30000)),#500000
    'validation': tokenized_datasets['validation'].select(np.arange(20000))#5000
}

In [3]:
tokenized_datasets['validation']['input_ids'][0]

[32565,
 13,
 15899,
 2497,
 262,
 22441,
 1097,
 290,
 531,
 11,
 366,
 22017,
 11,
 21168,
 11,
 534,
 1097,
 318,
 523,
 6016,
 290,
 3424,
 2474,
 21168,
 13541,
 290,
 8712,
 11,
 366,
 10449,
 345,
 11,
 15899,
 13,
 314,
 25245,
 340,
 790,
 1110,
 526,
 198,
 198,
 3260,
 2712,
 351,
 262,
 1097,
 11,
 21168,
 290,
 15899,
 2936,
 47124,
 13,
 1119,
 1043,
 257,
 1402,
 16723,
 351,
 1598,
 1660,
 13,
 1119,
 24070,
 262,
 1660,
 290,
 2936,
 845,
 3772,
 13,
 1119,
 2826,
 1978,
 477,
 1110,
 290,
 2627,
 1266,
 2460,
 25970,
 82,
 29,
 7454,
 2402,
 257,
 640,
 11,
 287,
 257,
 1263,
 8222,
 11,
 612,
 5615,
 257,
 9529,
 259,
 420,
 27498,
 3706,
 371,
 23536,
 13,
 371,
 23536,
 6151,
 284,
 12080,
 13,
 1375,
 19952,
 7150,
 11,
 12586,
 11,
 290,
 18639,
 13,
 1881,
 1110,
 11,
 371,
 23536,
 1043,
 281,
 30284,
 12788,
 13,
 1375,
 550,
 1239,
 1775,
 1997,
 588,
 340,
 878,
 13,
 632,
 373,
 22441,
 290,
 4692,
 11,
 290,
 673,
 2227,
 284,
 12080,
 340,
 13,
 198,
 198

In [19]:

ave = 0
for i in range(0,10):
    #input_str = tokenized_datasets['train']['input_ids'][i]
    #input_ids = tokenizer(input_str, return_tensors='pt')['input_ids']
    input_ids = torch.tensor(tokenized_datasets['validation']['input_ids'][3500 + i*1:3500 + (i+1)*1]).to('cuda')
    print('input id shape is ', input_ids.shape)
    model2 = model2.to('cuda')
    model2.eval()
    with torch.no_grad():
        print('im here!')
        loss_val = model2(input_ids, labels=input_ids)['loss']
        ave += loss_val
        print('i is ', i, ' loss is ', loss_val)
ave = ave / 10
print('average loss is ', ave)

input id shape is  torch.Size([1, 137])
im here!
i is  0  loss is  tensor(0.2377, device='cuda:0')
input id shape is  torch.Size([1, 261])
im here!
i is  1  loss is  tensor(0.3605, device='cuda:0')
input id shape is  torch.Size([1, 215])
im here!
i is  2  loss is  tensor(0.2397, device='cuda:0')
input id shape is  torch.Size([1, 158])
im here!
i is  3  loss is  tensor(0.2233, device='cuda:0')
input id shape is  torch.Size([1, 163])
im here!
i is  4  loss is  tensor(0.2123, device='cuda:0')
input id shape is  torch.Size([1, 129])
im here!
i is  5  loss is  tensor(0.2197, device='cuda:0')
input id shape is  torch.Size([1, 204])
im here!
i is  6  loss is  tensor(0.1964, device='cuda:0')
input id shape is  torch.Size([1, 209])
im here!
i is  7  loss is  tensor(0.2159, device='cuda:0')
input id shape is  torch.Size([1, 208])
im here!
i is  8  loss is  tensor(0.1898, device='cuda:0')
input id shape is  torch.Size([1, 187])
im here!
i is  9  loss is  tensor(0.1670, device='cuda:0')
average lo

In [17]:
## just testing the training loss is small
input_str = tinystories['train']['text'][130]
#input_str = ''' Once upon a time, there was a thoughtful girl named Sue. Sue loved tohelp her mom around the house. One day, her mom asked her to wipe thetable after they ate their lunch. Sue was happy to help. As Sue waswiping the table, she saw a pretty candle on the window sill. Thecandle was her mom's favorite. Sue wanted to do something nice for hermom, so she said, "Mom, can I light the candle for you?" Her mom said,"Yes, but be very careful." Sue carefully lit the candle and put it onthe table. Her mom was so happy to see the pretty candle. They bothsat and watched the candle burn. Sue's mom said, "Thank you, Sue, forbeing so thoughtful and careful." Sue felt proud that she could helpher mom. The moral of the story is to always be thoughtful and carefulwhen helping others.CLOSE TEXT IS:  Once upon a time, there was a little girl named Mia. She loved to helpher mom in the kitchen. One day, her mom asked her to unpack a bigbox. Mia was very happy to help. Inside the box, there was a newfaucet for the kitchen sink. Mia's mom said, "This faucet is veryspecial. It can go high and low." Mia watched as her mom put the newfaucet on the sink. When it was high, the water came out fast. When itwas low, the water came out slow. Mia learned that helping her mom wasfun and important. She felt proud when she saw the new faucet working.The moral of the story is that helping others is a good thing to do.'''

model2.eval()
with torch.no_grad():
    input_ids = tokenizer(input_str, return_tensors='pt')['input_ids']
    print('input_id length is ', input_ids.shape)
    hidden_embedding = model2.encoder(input_ids).last_hidden_state.detach()[0, -1, :].unsqueeze(0)
    decoder_hidden_inputs = model2.second_encoder(input_ids, output_hidden_states=True).hidden_states[0][0].detach()
    updated_input = torch.cat((hidden_embedding, decoder_hidden_inputs), dim=0)
    print(model2.decoder(inputs_embeds=updated_input, labels=torch.cat((torch.tensor([10]), input_ids[0])))['loss'])

input_id length is  torch.Size([1, 173])
tensor(3.8438)


In [ ]:
tinystories['train']['text'][10].shape

torch.Size([1, 173])

In [13]:
#model.decoder.lm_head.weight


Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)